In [10]:
import tensorflow as tf
import numpy as np
import pickle
import tensorflow.examples.tutorials.mnist.input_data as mnist
from tensorflow.examples.tutorials.mnist import input_data

In [11]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [12]:
def set_input():
    x = tf.placeholder(tf.float32, [None, 784], name="x_in")
    y_ = tf.placeholder("float", [None, 10], name="y_out")
    x_image = tf.reshape(x, [-1,28,28,1])
    return x,y_,x_image


## structure

In [13]:
def model_C(X):
    with tf.variable_scope("classifier"): 
        w_init = tf.truncated_normal_initializer(mean=0.0, stddev=0.1)
        b_init = tf.constant_initializer(0.0)

        # layer1
        conv1 = tf.layers.conv2d(X, filters=32, kernel_size=5, strides=1, padding="same",kernel_initializer=w_init,bias_initializer=b_init)
        conv1 = tf.nn.relu(conv1)
        pool1 = tf.layers.max_pooling2d(conv1, pool_size=2, strides=2, padding="same")
        
        # layer2
        conv2 = tf.layers.conv2d(pool1, filters=64, kernel_size=5, strides=1, padding="same",kernel_initializer=w_init,bias_initializer=b_init)
        conv2 = tf.nn.relu(conv2)
        pool2 = tf.layers.max_pooling2d(conv2,pool_size=2, strides=2)
        
        # layer3 FC
        data_flatten = tf.reshape(pool2,(-1, 7*7*64))
        dense1 = tf.layers.dense(data_flatten, 1024 )
        dense1 = tf.layers.dropout(dense1, rate=0.5)
        dense2 = tf.layers.dense(dense1, 10)
        
        out_model = tf.nn.softmax(dense2)
        
    return out_model

In [14]:
def loss(y, out_model):
    return -tf.reduce_sum(y_ * tf.log(out_model))

def optimize(loss_tmin):
    loss_optimize = tf.train.AdamOptimizer(1e-4).minimize(loss_tmin)
    return loss_optimize

def get_acc(y, out_model):
    state_y = tf.equal(tf.argmax(y,1), tf.argmax(out_model,1))
    accuracy = tf.reduce_mean(tf.cast(state_y, "float"))
    return accuracy

In [15]:
print(x.graph)

# build graph

In [31]:
tf.reset_default_graph()
x,y_,x_image = set_input()
out_model = model_C(x_image)
loss_model = loss(y_, out_model)
sess_optimize = optimize(loss_model)

# acc
state_y = tf.equal(tf.argmax(y_,1), tf.argmax(out_model,1))
accuracy = tf.reduce_mean(tf.cast(state_y, "float"))
# accuracy = get_acc(y_, out_model)


In [32]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction=0.7

In [33]:
batch = mnist.train.next_batch(50)
batch[1].shape

(50, 10)

In [41]:
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        batch = mnist.train.next_batch(64)
#     a = sess.run(out_model,feed_dict={x: batch[0]})
#     print(a.shape)
        _ = sess.run(sess_optimize,feed_dict={x: batch[0], y_:batch[1]})
        if i%100==1:
            train_accuracy = accuracy.eval(feed_dict={
               x: batch[0], y_: batch[1]})
            print("step %d, training accuracy %g" % (i, train_accuracy))
    print("test accuracy %g" % accuracy.eval(
    feed_dict={x: mnist.test.images[0:100], y_: mnist.test.labels[0:100]}))  


step 1, training accuracy 0.296875
step 101, training accuracy 0.953125
step 201, training accuracy 0.953125
step 301, training accuracy 0.953125
step 401, training accuracy 0.984375
step 501, training accuracy 0.96875
step 601, training accuracy 1
step 701, training accuracy 1
step 801, training accuracy 0.953125
step 901, training accuracy 1
step 1001, training accuracy 0.984375
step 1101, training accuracy 0.984375
step 1201, training accuracy 1
step 1301, training accuracy 1
step 1401, training accuracy 0.984375
step 1501, training accuracy 0.96875
step 1601, training accuracy 0.96875
step 1701, training accuracy 0.984375
step 1801, training accuracy 1
step 1901, training accuracy 0.96875
test accuracy 1


In [42]:
mnist.test.images[0:100].shape

(100, 784)

In [46]:
with open('samples_DCgan_final.pkl', 'rb') as f:
    pic_generate = pickle.load(f)
    
pic_in = pic_generate.reshape(-1,28*28)
pic_in.shape

(100, 784)

In [53]:
with open('label_DCgan.pkl','rb') as f:
    pic_label = pickle.load(f)


pic_label_1h = np.zeros((100,10))
pic_label_1h[np.arange(100), pic_label] = 1

pic_label_1h.shape

(100, 10)

In [57]:

with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch = mnist.train.next_batch(64)
#     a = sess.run(out_model,feed_dict={x: batch[0]})
#     print(a.shape)
        _ = sess.run(sess_optimize,feed_dict={x: batch[0], y_:batch[1]})
        if i%100==1:
            train_accuracy = accuracy.eval(feed_dict={
               x: batch[0], y_: batch[1]})
            print("step %d, training accuracy %g" % (i, train_accuracy))
        if i%500==1:
            print("test accuracy %g" % accuracy.eval(
            feed_dict={x: pic_in, y_: pic_label_1h}))  

step 1, training accuracy 0.28125
test accuracy 0.1
step 101, training accuracy 0.953125
step 201, training accuracy 0.953125
step 301, training accuracy 0.96875
step 401, training accuracy 0.96875
step 501, training accuracy 0.921875
test accuracy 0.31
step 601, training accuracy 1
step 701, training accuracy 0.96875
step 801, training accuracy 0.984375
step 901, training accuracy 0.96875
step 1001, training accuracy 0.984375
test accuracy 0.37
step 1101, training accuracy 0.984375
step 1201, training accuracy 1
step 1301, training accuracy 0.953125
step 1401, training accuracy 0.96875
step 1501, training accuracy 1
test accuracy 0.38
step 1601, training accuracy 1
step 1701, training accuracy 1
step 1801, training accuracy 1
step 1901, training accuracy 0.984375
step 2001, training accuracy 1
test accuracy 0.41
step 2101, training accuracy 0.984375
step 2201, training accuracy 0.984375
step 2301, training accuracy 1
step 2401, training accuracy 1
step 2501, training accuracy 1
test a

In [61]:
def model_C_regu(X):
    with tf.variable_scope("classifier"): 
        w_init = tf.truncated_normal_initializer(mean=0.0, stddev=0.1)
        b_init = tf.constant_initializer(0.0)

        # layer1
        conv1 = tf.layers.conv2d(X, filters=32, kernel_size=5, strides=1, padding="same",kernel_initializer=w_init,bias_initializer=b_init,
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(0.1))
        conv1 = tf.nn.relu(conv1)
        
        pool1 = tf.layers.max_pooling2d(conv1, pool_size=2, strides=2, padding="same")
        pool1 = tf.layers.batch_normalization(pool1, training=True)
        # layer2
        conv2 = tf.layers.conv2d(pool1, filters=64, kernel_size=5, strides=1, padding="same",kernel_initializer=w_init,bias_initializer=b_init,
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(0.1))
        conv2 = tf.nn.relu(conv2)
        
        pool2 = tf.layers.max_pooling2d(conv2,pool_size=2, strides=2)
        pool2 = tf.layers.batch_normalization(pool2, training=True)
        # layer3 FC
        data_flatten = tf.reshape(pool2,(-1, 7*7*64))
        dense1 = tf.layers.dense(data_flatten, 1024,
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(0.1))
        dense1 = tf.layers.dropout(dense1, rate=0.5)
        dense2 = tf.layers.dense(dense1, 10)
        
        out_model = tf.nn.softmax(dense2)
        
    return out_model

In [62]:
tf.reset_default_graph()
x,y_,x_image = set_input()
out_model = model_C_regu(x_image)
loss_model = loss(y_, out_model)
sess_optimize = optimize(loss_model)

# acc
state_y = tf.equal(tf.argmax(y_,1), tf.argmax(out_model,1))
accuracy = tf.reduce_mean(tf.cast(state_y, "float"))
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10000):
        batch = mnist.train.next_batch(64)
#     a = sess.run(out_model,feed_dict={x: batch[0]})
#     print(a.shape)
        _ = sess.run(sess_optimize,feed_dict={x: batch[0], y_:batch[1]})
        if i%100==1:
            train_accuracy = accuracy.eval(feed_dict={
               x: batch[0], y_: batch[1]})
            print("step %d, training accuracy %g" % (i, train_accuracy))
        if i%500==1:
            print("test accuracy %g" % accuracy.eval(
            feed_dict={x: pic_in, y_: pic_label_1h}))  

step 1, training accuracy 0.4375
test accuracy 0.18
step 101, training accuracy 0.984375
step 201, training accuracy 0.953125
step 301, training accuracy 0.953125
step 401, training accuracy 0.96875
step 501, training accuracy 0.984375
test accuracy 0.55
step 601, training accuracy 0.984375
step 701, training accuracy 1
step 801, training accuracy 0.984375
step 901, training accuracy 0.984375
step 1001, training accuracy 0.96875
test accuracy 0.56
step 1101, training accuracy 0.96875
step 1201, training accuracy 0.984375
step 1301, training accuracy 1
step 1401, training accuracy 0.984375
step 1501, training accuracy 1
test accuracy 0.64
step 1601, training accuracy 1
step 1701, training accuracy 1
step 1801, training accuracy 0.96875
step 1901, training accuracy 1
step 2001, training accuracy 0.984375
test accuracy 0.61
step 2101, training accuracy 0.984375
step 2201, training accuracy 1
step 2301, training accuracy 0.984375
step 2401, training accuracy 0.96875
step 2501, training acc

In [65]:
def model_C_regu_dropconv(X):
    with tf.variable_scope("classifier"): 
        w_init = tf.truncated_normal_initializer(mean=0.0, stddev=0.1)
        b_init = tf.constant_initializer(0.0)

        # layer1
        conv1 = tf.layers.conv2d(X, filters=32, kernel_size=5, strides=1, padding="same",kernel_initializer=w_init,bias_initializer=b_init,
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(0.1))
        conv1 = tf.nn.relu(conv1)
        conv1 = tf.layers.dropout(conv1, rate=0.2)
        
        pool1 = tf.layers.max_pooling2d(conv1, pool_size=2, strides=2, padding="same")
        pool1 = tf.layers.batch_normalization(pool1, training=True)
        # layer2
        conv2 = tf.layers.conv2d(pool1, filters=64, kernel_size=5, strides=1, padding="same",kernel_initializer=w_init,bias_initializer=b_init,
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(0.1))
        conv2 = tf.nn.relu(conv2)
        conv2 = tf.layers.dropout(conv2, rate=0.2)
        
        pool2 = tf.layers.max_pooling2d(conv2,pool_size=2, strides=2)
        pool2 = tf.layers.batch_normalization(pool2, training=True)
        # layer3 FC
        data_flatten = tf.reshape(pool2,(-1, 7*7*64))
        dense1 = tf.layers.dense(data_flatten, 1024,
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(0.1))
        dense1 = tf.layers.dropout(dense1, rate=0.5)
        dense2 = tf.layers.dense(dense1, 10)
        
        out_model = tf.nn.softmax(dense2)
        
    return out_model

In [66]:
tf.reset_default_graph()
x,y_,x_image = set_input()
out_model = model_C_regu_dropconv(x_image)
loss_model = loss(y_, out_model)
sess_optimize = optimize(loss_model)

# acc
state_y = tf.equal(tf.argmax(y_,1), tf.argmax(out_model,1))
accuracy = tf.reduce_mean(tf.cast(state_y, "float"))
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(5000):
        batch = mnist.train.next_batch(64)
#     a = sess.run(out_model,feed_dict={x: batch[0]})
#     print(a.shape)
        _ = sess.run(sess_optimize,feed_dict={x: batch[0], y_:batch[1]})
        if i%100==1:
            train_accuracy = accuracy.eval(feed_dict={
               x: batch[0], y_: batch[1]})
            print("step %d, training accuracy %g" % (i, train_accuracy))
        if i%500==1:
            print("test accuracy %g" % accuracy.eval(
            feed_dict={x: pic_in, y_: pic_label_1h}))  

step 1, training accuracy 0.453125
test accuracy 0.17
step 101, training accuracy 0.96875
step 201, training accuracy 0.96875
step 301, training accuracy 0.96875
step 401, training accuracy 0.96875
step 501, training accuracy 0.96875
test accuracy 0.61
step 601, training accuracy 0.953125
step 701, training accuracy 0.953125
step 801, training accuracy 0.984375
step 901, training accuracy 0.96875
step 1001, training accuracy 1
test accuracy 0.64
step 1101, training accuracy 0.984375
step 1201, training accuracy 0.984375
step 1301, training accuracy 1
step 1401, training accuracy 1
step 1501, training accuracy 0.984375
test accuracy 0.65
step 1601, training accuracy 0.984375
step 1701, training accuracy 1
step 1801, training accuracy 0.984375
step 1901, training accuracy 1
step 2001, training accuracy 1
test accuracy 0.71
step 2101, training accuracy 0.953125
step 2201, training accuracy 0.984375
step 2301, training accuracy 1
step 2401, training accuracy 1
step 2501, training accuracy 

In [71]:
def model_C_regu_dropconv_m(X):
    with tf.variable_scope("classifier"): 
        w_init = tf.truncated_normal_initializer(mean=0.0, stddev=0.1)
        b_init = tf.constant_initializer(0.0)

        # layer1
        conv1 = tf.layers.conv2d(X, filters=32, kernel_size=5, strides=1, padding="same",kernel_initializer=w_init,bias_initializer=b_init,
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
        conv1 = tf.nn.relu(conv1)
        conv1 = tf.layers.dropout(conv1, rate=0.2)
        
        pool1 = tf.layers.max_pooling2d(conv1, pool_size=2, strides=2, padding="same")
        pool1 = tf.layers.batch_normalization(pool1, training=True)
        # layer2
        conv2 = tf.layers.conv2d(pool1, filters=64, kernel_size=5, strides=1, padding="same",kernel_initializer=w_init,bias_initializer=b_init,
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
        conv2 = tf.nn.relu(conv2)
        conv2 = tf.layers.dropout(conv2, rate=0.2)
        
        pool2 = tf.layers.max_pooling2d(conv2,pool_size=2, strides=2)
        pool2 = tf.layers.batch_normalization(pool2, training=True)
        # layer3 FC
        data_flatten = tf.reshape(pool2,(-1, 7*7*64))
        dense1 = tf.layers.dense(data_flatten, 1024,
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(0.2))
        dense1 = tf.layers.dropout(dense1, rate=0.5)
        dense2 = tf.layers.dense(dense1, 10)
        
        out_model = tf.nn.softmax(dense2)
        
    return out_model

In [72]:
tf.reset_default_graph()
x,y_,x_image = set_input()
out_model = model_C_regu_dropconv_m(x_image)
loss_model = loss(y_, out_model)
sess_optimize = optimize(loss_model)

# acc
state_y = tf.equal(tf.argmax(y_,1), tf.argmax(out_model,1))
accuracy = tf.reduce_mean(tf.cast(state_y, "float"))
with tf.Session(config=config) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(4501):
        batch = mnist.train.next_batch(64)
#     a = sess.run(out_model,feed_dict={x: batch[0]})
#     print(a.shape)
        _ = sess.run(sess_optimize,feed_dict={x: batch[0], y_:batch[1]})
        if i%100==1:
            train_accuracy = accuracy.eval(feed_dict={
               x: batch[0], y_: batch[1]})
            print("step %d, training accuracy %g" % (i, train_accuracy))
        if i%500==1:
            print("test accuracy %g" % accuracy.eval(
            feed_dict={x: pic_in, y_: pic_label_1h}))  

step 1, training accuracy 0.4375
test accuracy 0.16
step 101, training accuracy 0.984375
step 201, training accuracy 1
step 301, training accuracy 1
step 401, training accuracy 0.96875
step 501, training accuracy 0.984375
test accuracy 0.57
step 601, training accuracy 0.953125
step 701, training accuracy 0.96875
step 801, training accuracy 0.984375
step 901, training accuracy 1
step 1001, training accuracy 1
test accuracy 0.55
step 1101, training accuracy 0.953125
step 1201, training accuracy 1
step 1301, training accuracy 1
step 1401, training accuracy 0.953125
step 1501, training accuracy 1
test accuracy 0.57
step 1601, training accuracy 0.984375
step 1701, training accuracy 1
step 1801, training accuracy 1
step 1901, training accuracy 1
step 2001, training accuracy 0.96875
test accuracy 0.69
step 2101, training accuracy 1
step 2201, training accuracy 1
step 2301, training accuracy 1
step 2401, training accuracy 0.96875
step 2501, training accuracy 1
test accuracy 0.63
step 2601, tra